# Implementing the Morais-Lima-Martin (MLM) Sampler
The notebook shows a brief demonstration of using the built in utilities in `astartes` to implement the Morais-Lima-Martin sampler, which you can read about [here](https://academic.oup.com/bioinformatics/article/35/24/5257/5497250).

`astartes` has a very fast implementation of the Kennard-Stone algorithm, on which the MLM sampler is based, available in its `utils`:

In [ ]:
from astartes.utils.fast_kennard_stone import fast_kennard_stone

The MLM sampler can then be implemented as shown below.
The `mlm_sampler` functions takes a 2D array and splits it first using the Kennard-Stone algorithm, then permutes the indices according to the MLM algorithm.

In [ ]:
from scipy.spatial.distance import pdist, squareform
import numpy as np

from astartes.samplers.interpolation import KennardStone


def mlm_split(X: np.ndarray, *, train_size: float = 0.8, val_size: float = 0.1, test_size: float = 0.1, random_state: int = 42):
    # calculate the distance matrix
    ks_indexes = fast_kennard_stone(squareform(pdist(X, "euclidean")))
    pivot = int(len(ks_indexes) * train_size)
    train_idxs = ks_indexes[0:pivot]
    other_idxs = ks_indexes[pivot:]

    # set RNG
    rng = np.random.default_rng(seed=random_state)
    
    # choose 10% of train to switch with 10% of val/test
    n_to_permute = np.floor(0.1 * len(train_idxs))
    train_permute_idxs = rng.choice(train_idxs, n_to_permute)
    remaining_train_idxs = filter(lambda i: i not in train_permute_idxs, train_idxs)
    other_permute_idxs = rng.choice(other_idxs, n_to_permute)
    remaining_other_idxs = filter(lambda i: i not in other_permute_idxs, other_idxs)

    # reassemble the new lists of indexes
    new_train_idxs = np.concatenate(remaining_train_idxs, other_permute_idxs)
    new_other_idxs = np.concatenate(train_permute_idxs, remaining_other_idxs)
    n_val = int(len(new_other_idxs) * (val_size / (val_size + test_size)))
    val_indexes = new_other_idxs[0:n_val]
    test_indexes = new_other_idxs[n_val:]
    
    # return the split up array
    return X[train_idxs], X[val_indexes], X[test_indexes]
    